# 1. Data preprocessing

Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
* взять в тренировочный набор турниры с dateStart из 2019 года; 
* в тестовый — турниры с dateStart из 2020 года.


In [43]:
# Импорт библиотек
import pickle
import numpy as np
import scipy as sp
import pandas as pd
import tqdm
from scipy import stats

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVR
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
# Загружаем данные
players = pd.read_pickle("data/players.pkl")
results = pd.read_pickle("data/results.pkl")  
tournaments = pd.read_pickle("data/tournaments.pkl")  


In [3]:
players = pd.DataFrame(players).T
tournaments = pd.DataFrame(tournaments).T

In [4]:
idx_list_test = tournaments[tournaments['dateStart'] >= '2020'].index
idx_list_train = tournaments[tournaments['dateStart'] >= '2019'][tournaments['dateStart']  <'2020'].index
print('Data train length: ', len(idx_list_train))
print('Data test length: ', len(idx_list_test))


Data train length:  687
Data test length:  422


<ipython-input-4-9e2f536672b2>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idx_list_train = tournaments[tournaments['dateStart'] >= '2019'][tournaments['dateStart']  <'2020'].index


In [5]:
# Отберем турниры с данными по team и mask
def idx_clean(idx_list):
    new_idx_list = []
    for i in idx_list:
        try:
            if results[i][0]['team'] and results[i][0]['mask']:
                new_idx_list.append(i)
        except:
            continue
    return new_idx_list

# Aктуальные списки соревнований    
idx_list_test = idx_clean(idx_list_test)
idx_list_train = idx_clean(idx_list_train)

print('Clean data train length: ', len(idx_list_train))
print('Clean data test length: ', len(idx_list_test))


Clean data train length:  674
Clean data test length:  173


In [6]:
def create_df(idx_list):
    df_results = []
    for idx in tqdm.tqdm(idx_list):
        for team in results[idx]:
            if team['mask']:
                mask = str(team['mask']).replace('X','0').replace('?', '0')
                players = team['teamMembers']
                team_id = team['team']['id']
                for player in players:  
                    player_id = player['player']['id']
                    for no_q, answer in enumerate(mask): 
                        df_results.append([idx, team_id, player_id, no_q, answer])
    df = pd.DataFrame(df_results)
    df.columns = ['tournament_id', 'team_id', 'player_id', 'question', 'answer']
    return df

df_train = create_df(idx_list_train)
df_test = create_df(idx_list_test)

100%|████████████████████████████████████████████████████████████████████████████████| 173/173 [00:28<00:00,  6.08it/s]


In [7]:
# Соберем разметку позиций с сайта ЧГК? в словарик, для удобства обращения
dict_labels = {}
for idx in tqdm.tqdm(idx_list_test):
    dict_labels[idx] = {}
    for team in results[idx]:
        team_id = team['team']['id']
        team_pos = team['position']
        dict_labels[idx][team_id] = team_pos

100%|██████████████████████████████████████████████████████████████████████████████| 173/173 [00:00<00:00, 8774.16it/s]


In [8]:
# Сохраняем данные
df_train.to_pickle("data/df_train.pkl") 
df_test.to_pickle("data/df_test.pkl") 
pickle.dump(dict_labels, open('data/dict_labels', 'wb'))

# 2. Baseline-model

 Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
* повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
* в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
* для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


In [9]:
%reload_ext autoreload
%autoreload 2

In [10]:
# Загружаем данные
df_train = pickle.load(open('data/df_train.pkl', 'rb'))

In [11]:
n_players = df_train.player_id.nunique()
n_tourn = df_train.tournament_id.nunique()
print('number of players = ', n_players)
print('number of tournamets = ', n_tourn)

number of players =  59100
number of tournamets =  674


Обучим логистическую регрессию на Dummy-переменных, т.е. переведем в признаки всех игроков, а также турниры. Тем самым в коэффициентах регрессии получим силы игроков, а также сложности потурнирных вопросов.

In [12]:
categorical_transformer = OneHotEncoder()
categorical_features = ['tournament_id', 'player_id']
X, y = df_train[categorical_features], df_train['answer']

preprocessor = ColumnTransformer(
    transformers=[
        ("Dummies", categorical_transformer, categorical_features),
    ]
)

clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression(solver='liblinear'))]
)

clf.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('Dummies', OneHotEncoder(),
                                                  ['tournament_id',
                                                   'player_id'])])),
                ('classifier', LogisticRegression(solver='liblinear'))])

In [110]:
powers = clf['classifier'].coef_[0][n_tourn:]

Создадим табличку сил игроков

In [111]:
players_ids = sorted(df_train.player_id.unique())
players_rating = dict(zip(players_ids, powers))

In [15]:
# Сохраняем данные
pickle.dump(clf, open('data/clf_baseline', 'wb'))
pickle.dump(players_rating, open('data/players_rating', 'wb'))

# 3. Validation
Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
* предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
* в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

Для самопроверки: у Cергея средняя корреляция Спирмена на тестовом множестве 2020 года во всех моделях, включая baselines, получалась порядка 0.7-0.8, а корреляция Кендалла — порядка 0.5-0.6. Если у вас корреляции вышли за 0.9 или, наоборот, упали ниже 0.3, скорее всего где-то баг.

In [16]:
%reload_ext autoreload
%autoreload 2

In [17]:
# Загружаем данные
df_train = pickle.load(open('data/df_train.pkl', 'rb'))
df_test = pickle.load(open('data/df_test.pkl', 'rb'))
clf = pickle.load(open('data/clf_baseline', 'rb'))
players_rating = pickle.load(open('data/players_rating', 'rb'))

In [18]:
df = df_test.groupby(['tournament_id','team_id','player_id']).question.count().reset_index()
df.head()

,tournament_id,team_id,player_id,question
0,4957,2,6482,39
1,4957,2,25882,39
2,4957,2,30475,39
3,4957,2,32458,39
4,4957,2,34846,39


In [19]:
# Добавим в таблицу столбец с рейтингом игроков
df['player_rating'] = df['player_id'].apply(lambda x: players_rating[x] if x in players_rating.keys() else np.nan)

In [20]:
# Сгруппируем данные по турнирам и командам и проссумируем покомандные рейтинги
team_raiting = df.groupby(['tournament_id','team_id'])['player_rating'].sum().reset_index()
team_raiting.columns = ['tournament_id','team_id', 'rating']
team_raiting = team_raiting.groupby(['tournament_id','rating','team_id']).count().reset_index()

In [21]:
# Посчитаем по суммарным рейтингам команд их позиции на турнире. Просто отсортируем рейтинги и присвоим командам места.
labels_pred = []
for t in team_raiting.tournament_id.unique():
    n = len(team_raiting[team_raiting['tournament_id'] == t])
    for i in range(n):
        labels_pred.append(n-i)
team_raiting['labels_pred'] = pd.Series(labels_pred)

In [22]:
# Соберем разметку позиций с сайта ЧГК? в словарик, для удобства обращения
dict_labels = {}
for idx in tqdm.tqdm(idx_list_test):
    dict_labels[idx] = {}
    for team in results[idx]:
        team_id = team['team']['id']
        team_pos = team['position']
        dict_labels[idx][team_id] = team_pos


100%|███████████████████████████████████████████████████████████████████████████████| 173/173 [00:00<00:00, 264.36it/s]


In [23]:
# Объединим все в кучу для наглядности
labels = []
for i in team_raiting.index:
    tournament = team_raiting['tournament_id'][i]
    team_id = team_raiting['team_id'][i]
    labels.append(dict_labels[tournament][team_id])
team_raiting['labels'] = pd.Series(labels)
team_raiting.head()

,tournament_id,rating,team_id,labels_pred,labels
0,4957,-17.541271,49804,92,1.0
1,4957,-14.671829,77418,91,4.0
2,4957,-14.604966,2,90,5.5
3,4957,-14.318287,46381,89,31.5
4,4957,-14.290825,27177,88,22.0


In [24]:
# Посчитаем корреляции отдельно по турнирам, а затем посмотрим на среднее.
Spearman = []
Kendall = []
for tournament in team_raiting.tournament_id.unique():
    df_tourn = team_raiting[team_raiting.tournament_id == tournament]
    if len(df_tourn) > 1:
        Spearman.append((stats.spearmanr(df_tourn['labels_pred'], df_tourn['labels']).correlation))
        Kendall.append(stats.kendalltau(df_tourn['labels_pred'], df_tourn['labels']).correlation)
print('Spearman correlation = ', round(np.mean(Spearman), 2))
print('Kendall correlation = ', round(np.mean(Kendall), 2))

Spearman correlation =  -0.78
Kendall correlation =  -0.62


In [56]:
# Объединим всю валидацию в одну функцию с определением метрик корреляции
players_ids = sorted(df_train.player_id.unique())
df = df_test.groupby(['tournament_id','team_id','player_id']).question.count().reset_index()
n_tourn = df_train.tournament_id.nunique()
def validation(clf, df, players_ids, dict_labels, n_tourn):
    # Создаем словарь с силой игрока
    powers = clf['classifier'].coef_[0][n_tourn:]
    players_rating = dict(zip(players_ids, powers))

    # Добавим в таблицу столбец с рейтингом игроков
    df['player_rating'] = df['player_id'].apply(lambda x: players_rating[x] if x in players_rating.keys() else np.nan)

    # Сгруппируем данные по турнирам и командам и проссумируем покомандные рейтинги
    team_raiting = df.groupby(['tournament_id','team_id'])['player_rating'].sum().reset_index()
    team_raiting.columns = ['tournament_id','team_id', 'rating']
    team_raiting = team_raiting.groupby(['tournament_id','rating','team_id']).count().reset_index()
    # Посчитаем по суммарным рейтингам команд их позиции на турнире. Просто отсортируем рейтинги и присвоим командам места.

    labels_pred = []
    for t in team_raiting.tournament_id.unique():
        n = len(team_raiting[team_raiting['tournament_id'] == t])
        for i in range(n):
            labels_pred.append(n-i)
    team_raiting['labels_pred'] = pd.Series(labels_pred)

     # Объединим все в кучу для наглядности
    labels = []
    for i in team_raiting.index:
        tournament = team_raiting['tournament_id'][i]
        team_id = team_raiting['team_id'][i]
        labels.append(dict_labels[tournament][team_id])
    team_raiting['labels'] = pd.Series(labels)

    # Посчитаем корреляции отдельно по турнирам, а затем посмотрим на среднее.
    Spearman = []
    Kendall = []
    for tournament in team_raiting.tournament_id.unique():
        df_tourn = team_raiting[team_raiting.tournament_id == tournament]
        if len(df_tourn) > 1:
            Spearman.append((stats.spearmanr(df_tourn['labels_pred'], df_tourn['labels']).correlation))
            Kendall.append(stats.kendalltau(df_tourn['labels_pred'], df_tourn['labels']).correlation)
    print('Spearman correlation = ', round(np.mean(Spearman), 2))
    print('Kendall correlation = ', round(np.mean(Kendall), 2))

# 4. EM algorithm
Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
разработайте EM-схему для обучения этой модели, реализуйте её в коде;
обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.


В качестве скрытых переменных будем использовать ответы игроков на каждый вопрос $z_{iq}$ - икгрок i ответил на вопрос q. Cчитаем, что команда ответила на вопрос, если хотябы один игрок из команды ответил на него правильно, и, соответственно, если никто из игроков не ответил на впорос, то и команда на него не ответила.

### E-шаг
Запоминаем предсказания логистической регрессии и вычисляем ожидания скрытых переменных

$$
 {\displaystyle \mathbb {E} [z_{iq}]} = \begin{cases}
 & 0, & \quad  \text{если } x_{tq} = 0,\\
 & \frac{p (z_{ij} = 1)}{1 - \Pi_{k \in t} (1 - p (z_{kj}))} & \quad  \text{если } x_{tq} = 1,
 \end{cases}
$$

Так как у нас бинарная классификация, то просто максимум из ответов игроков по команде будет по сути являться ответом всей команды.

In [25]:
%reload_ext autoreload
%autoreload 2

In [137]:
# Загружаем данные
df_train = pickle.load(open('data/df_train.pkl', 'rb'))
df_test = pickle.load(open('data/df_test.pkl', 'rb'))
clf = pickle.load(open('data/clf_baseline', 'rb'))
players_rating = pickle.load(open('data/players_rating', 'rb'))
dict_labels = pickle.load(open('data/dict_labels', 'rb'))

In [67]:
# Инициализируем логистическую регрессию
def train_model(X, y):
    categorical_transformer = OneHotEncoder()
    preprocessor = ColumnTransformer(
        transformers=[
            ("Dummies", categorical_transformer, categorical_features),
        ]
    )

    clf = Pipeline(
        steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression(solver='liblinear'))]
    )

    clf.fit(X, y)
    return clf

In [68]:
# Возьмем предсказания с нашей обученной модели
categorical_features = ['tournament_id', 'player_id']
X, y = df_train[categorical_features], df_train['answer']
clf = train_model(X, y)
y_pred = clf.predict(X)


In [70]:
def e_step(df, y_pred):
    df = df_train.copy()
    df['predict'] = y_pred
    # Зануляем веросятности игроков, где ответ команды 0
    df.loc[df['answer'] == '0','predict'] = 0
    df['predict'] = df['predict'].apply(lambda x: int(x))
    # Группируем данные по турниру, командам и вопросам и из предсказаний по членам команд выбираем максимальное значение
    y_new= df.groupby(['tournament_id', 'team_id', 'question'])['predict'].transform('max')
    return y_new

### M-шаг
Обучаем логистическую регрессию с $\mathbb {E}[z_{iq}]$

In [71]:
def m_step(clf, X, y):
    clf = train_model(X, y)
    return clf, clf.predict(X)

### Обучение EM - алгоритма

In [86]:
print('Epoch 0 (initialization)')
validation(clf, df, players_ids, dict_labels, n_tourn)

Epoch 0 (initialization)
Spearman correlation =  0.78
Kendall correlation =  0.62


In [72]:
n_epoch = 5
X, y = df_train[categorical_features], df_train['answer']
for epoch in range(n_epoch):
    print('Epoch ', epoch + 1)
    y = e_step(df_train, y_pred)
    clf, y_pred = m_step(clf, X, y)
    validation(clf, df, players_ids, dict_labels, n_tourn)
    # Сохраняем данные
    pickle.dump(clf, open('data/clf_EM_' + str( epoch + 1) + '_epoch', 'wb'))


Epoch  1
Spearman correlation =  0.74
Kendall correlation =  0.58
Epoch  2
Spearman correlation =  0.68
Kendall correlation =  0.53
Epoch  3
Spearman correlation =  0.65
Kendall correlation =  0.5
Epoch  4
Spearman correlation =  0.62
Kendall correlation =  0.47
Epoch  5
Spearman correlation =  0.6
Kendall correlation =  0.45


### Размышления:
От итерации EM алгоритма качество снижается. Я думаю, что это может быть все таки связано с тем, что нужно обобщать результаты игроков иначе, чем просто судить по одному игроку, который ответил правильно. Например перевзвешивать результаты с силой игроков и брать произведение вероятностей ответов всех игроков.

In [138]:
# Загружаем данные
df_train = pickle.load(open('data/df_train.pkl', 'rb'))
df_test = pickle.load(open('data/df_test.pkl', 'rb'))
clf = pickle.load(open('data/clf_baseline', 'rb'))
players_rating = pickle.load(open('data/players_rating', 'rb'))
dict_labels = pickle.load(open('data/dict_labels', 'rb'))

In [113]:
# Добавим в таблицу столбец с рейтингом игроков
df_train['player_rating'] = df_train['player_id'].apply(lambda x: players_rating[x] if x in players_rating.keys() else np.nan)

In [123]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
player_rating = scaler.fit_transform(df_train['player_rating'].values.reshape(-1, 1)).reshape(1, -1)[0]

In [ ]:
def e_step(df, y_pred, player_rating):
    df = df_train.copy()
    df['predict'] = player_rating * np.array([int(x) for x in y_pred])
    # Зануляем веросятности игроков, где ответ команды 0
    df.loc[df['answer'] == '0','predict'] = 0
#     df['predict'] = df['predict'].apply(lambda x: int(x))
    # Группируем данные по турниру, командам и вопросам и из предсказаний по членам команд выбираем суммарные значение
    y_new= df.groupby(['tournament_id', 'team_id', 'question'])['predict'].transform('sum').apply(lambda x: 1 if round(x)>=1 else 0)
    return y_new

def m_step(clf, X, y):
    clf = train_model(X, y)
    return clf, clf.predict(X)


In [ ]:
n_epoch = 5
X, y = df_train[categorical_features], df_train['answer']

print('Epoch 0 (initialization)')
validation(clf, df, players_ids, dict_labels, n_tourn)
y_pred = clf.predict(X)

for epoch in range(n_epoch):
    print('Epoch ', epoch + 1)
    y = e_step(df_train, y_pred, player_rating)
    clf, y_pred = m_step(clf, X, y)
    validation(clf, df, players_ids, dict_labels, n_tourn)
    # Сохраняем данные
    pickle.dump(clf, open('data/clf_EM_' + str( epoch + 1) + '_epoch', 'wb'))


Epoch 0 (initialization)
Spearman correlation =  0.78
Kendall correlation =  0.62
Epoch  1
Spearman correlation =  0.73
Kendall correlation =  0.57
Epoch  2
Spearman correlation =  0.68
Kendall correlation =  0.53
Epoch  3
Spearman correlation =  0.65
Kendall correlation =  0.5
Epoch  4
Spearman correlation =  0.62
Kendall correlation =  0.47
Epoch  5


Все равно что-то не оч)))) Но спускается корреляция меньше, чем при оценке ответа одного участника команды.

# 5. Rating list of tournaments question
А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).


In [73]:
clf_5 = pickle.load(open('data/clf_EM_' + str(5) + '_epoch', 'rb'))

In [77]:
level_q = -clf_5['classifier'].coef_[0][:n_tourn]

In [81]:
import operator
# Создадим словарь со сложностями турниров, отберем по 10 самых сложных и самых простых турниров
level_dict = dict(zip(sorted(df_train['tournament_id'].unique()), level_q))
sorted_tuples = sorted(level_dict.items(), key=operator.itemgetter(1), reverse=False)
Hard_top_10 = {k:v for k, v in sorted_tuples[:-10:-1]}
Easy_top_10 = {k:v for k, v in sorted_tuples[:10]}

In [82]:
Hardest_10 = {}
Easiest_10 = {}
for idx, level in Easy_top_10.items():
    Easiest_10[idx] = {}
    Easiest_10[idx]['name'] = tournaments.loc[idx]['name']
    Easiest_10[idx]['level'] = round(level, 2)
    
for idx, level in Hard_top_10.items():
    Hardest_10[idx] = {}
    Hardest_10[idx]['name'] = tournaments.loc[idx]['name']
    Hardest_10[idx]['level'] = round(level, 2)
    
    
print('Hard level top 10 ')
for key,value in Hardest_10.items(): 
    print(key, ':', value)
print()    
print('Easy level top 10 ')
for key,value in Easiest_10.items(): 
    print(key, ':', value)

Hard level top 10 
5465 : {'name': 'Чемпионат России', 'level': 7.54}
5025 : {'name': 'Кубок городов', 'level': 7.4}
5757 : {'name': 'Открытый Кубок России', 'level': 7.26}
5594 : {'name': 'Кубок чемпионов', 'level': 7.0}
5451 : {'name': 'Весна в ЛЭТИ', 'level': 7.0}
5516 : {'name': 'Синхрон Моносова', 'level': 6.91}
5864 : {'name': 'Гран-при Славянки. Общий зачет', 'level': 6.91}
5756 : {'name': 'Жизнь и время Михаэля К.', 'level': 6.89}
5795 : {'name': 'Кубок Москвы', 'level': 6.78}

Easy level top 10 
5457 : {'name': 'Студенческий чемпионат Калининградской области', 'level': -8.84}
5013 : {'name': '(а)Синхрон-lite. Лига старта. Эпизод V', 'level': -8.28}
5009 : {'name': '(а)Синхрон-lite. Лига старта. Эпизод III', 'level': -8.1}
5658 : {'name': 'Кубок Югры', 'level': -7.81}
5679 : {'name': 'Чемпионат Караганды', 'level': -7.76}
5438 : {'name': 'Синхрон Лиги Разума', 'level': -7.7}
5011 : {'name': '(а)Синхрон-lite. Лига старта. Эпизод IV', 'level': -7.49}
5698 : {'name': '(а)Синхрон-l

Обученные сложности логистической регрессией соответствуют сложностям турниров, что говорит об адекватном поведении модели